In [2]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import numpy as np

ModuleNotFoundError: No module named 'gurobipy'

In [2]:
data = pd.read_excel('Top.xlsx')
data = data[['구간', '연장(km)', '차로수', '계']]
data.head(10)

,지점번호,도명,시/군,읍/면,동/리,구간,이정(km),연장(km),차로수,계,승용차,버스,소형화물,중형화물,대형화물,Unnamed: 15
0,101,부산,NaN,금정,구서,구서IC-노포IC,0~5.1,5.1,6,71512,48589,2372,11959,5803,2789,NaN
1,00101-1,부산,NaN,금정,노포,노포IC-노포JCT,5.1~5.6,0.5,6,66817,48737,1690,10047,4141,2202,5.850
2,00101-2,부산,NaN,금정,노포,노포JCT-양산JCT,5.6~12.9,7.3,6,86906,67766,1776,13028,3225,1111,13.275
3,102,경남,양산,NaN,다방,양산JCT-양산IC,12.9~18,5.1,8,89018,62753,1549,14128,6988,3600,18.500
4,103,경남,양산,상북,석계,양산IC-통도사하이패스IC,18~30.8,12.8,6,71192,48125,1239,11835,6633,3360,31.425
5,00103-1,울산,울주,삼남,순지,통도사하이패스IC-통도사IC,30.8~32.2,1.4,6,68352,44452,1125,11931,7146,3698,32.950
6,104,울산,울주,삼남,조일,통도사IC-서울산IC,32.2~38.6,6.4,6,65305,38620,1136,11170,10127,4252,39.475
7,105,울산,울주,삼남,교,서울산IC-언양JCT,38.6~40.1,1.5,6,58585,33178,1049,9761,10448,4149,41.100
8,106,울산,울주,언양,반곡,언양JCT-활천IC,40.1~57.3,17.2,4,49423,31470,1006,7044,7429,2474,58.425
9,00106-1,경북,경주,내남,용장,활천IC-경주IC,57.3~68.2,10.9,4,45602,25095,747,7763,9004,2993,69.450


In [ ]:
def optimize(data) :
    try:

        # Create a new model
        m = gp.Model("qp")

        # Create variables
        for i in range(1, len(data)+1):
            globals()['x_%d'%i] = m.addVar(vtype=GRB.BINARY, name="x_%d"%i)
            globals()['y_%d'%i] = m.addVar(vtype=GRB.CONTINUOUS, name="y_%d"%i)
        for i in range(len(data)):
            globals()['l_%d'%(i+1)] = data.loc[i, '연장(km)']
            globals()['n_%d'%(i+1)] = data.loc[i, '차로수']
            globals()['e_%d'%(i+1)] = data.loc[i, '계']

        # Set objective
        obj = None
        for i in range(1, len(data)+1):
            obj += globals()['x_%d'%i] * globals()['y_%d'%i] * globals()['e_%d'%i]
        m.setObjective(obj, GRB.MAXIMIZE)

        const = None
        for i in range(1, len(data)+1):
            const +=  globals()['x_%d'%i] * globals()['y_%d'%i] * globals()['n_%d'%i]
        # Add constraint: x + 2 y + 3 z <= 4
        m.addConstr(13.5 * const <= 5179, "c0")
        
        
        # Add constraint: x + y >= 1
        for i in range(1, len(data)+1):
            m.addConstr(globals()['y_%d'%i] <= globals()['l_%d'%i], "c_%d"%i)

        # Optimize model
        m.optimize()

        print('Obj: %g' % m.objVal)
        return m

    except gp.GurobiError as e:
        print('Error code ' + str(e.errno) + ': ' + str(e))
        return None

    except AttributeError:
        print('Encountered an attribute error')
        return None

In [150]:
# Optimizing
project_year = 5
tmp = pd.DataFrame(data = [], columns = list(set(data.columns).union(set(['year']))))
year = 2022
next_year = []
for i in range(project_year):
    m = optimize(data)
    lst = [] 
    for v in m.getVars() :
        if "y" in v.varName :
            if v.x > 0 :
                lst.append(int(v.varName.split('_')[1])-1)
    a = data.loc[lst,]
    a['year'] = year
    year += 1
    tmp = pd.concat([a, tmp])
    tmp2 = []
    for i in lst :
        if data.iloc[i,1] != globals()['y_%d'%(i+1)].x :
            data.iloc[i,1] -= globals()['y_%d'%(i+1)].x
            tmp2.append(data.iloc[i,:])
            lst.remove(i)
    next_year.append(tmp2)
    data.drop(index = lst, inplace = True)
    data.reset_index(drop=True, inplace= True)
    data.to_csv('./Restopt{}.csv'.format(i))
tmp

In [1]:
import pandas as pd
import numpy as np

tmp = pd.read_csv('./tmp.csv')
data = pd.read_excel('Top.xlsx')

In [ ]:
idx = 0
for e in next_year:
    a = e[0]
    if idx == 0:
        ny = pd.DataFrame(a).T
    else:
        ny = pd.concat([ny, pd.DataFrame(a).T])
    idx += 1
ny

In [2]:
data = pd.read_excel('Top.xlsx')
data['지점번호'] = list(map(lambda x: int(x.split('-')[0]) if type(x)==type('1') else int(x), data['지점번호']))

lst = []
num = []
for e in sorted(list(set(tmp['year']))):
    df = data.merge(tmp[tmp['year']==e], how='inner', on=['구간', '연장(km)', '차로수', '계'])
    lst.append(list(map(lambda x: tuple(list(map(float, x.split('~')))),list(df['이정(km)']))))
    num.append(list(df['지점번호']))

ny = ny.merge(data, how='inner', on=['구간','차로수','계'])
ny['지점번호'] = list(map(lambda x: 10 if 100 <= x < 1000 else 500 if 5000 <= x < 9999 else 1000, data['지점번호']))

year = 2021
for i in range(len(lst)):
    year += 1
    if i == 0:
        data = pd.DataFrame(np.array([list(map(lambda x: x[0], lst[i])), list(map(lambda x: x[1], lst[i])), num[i]]).T, columns=['이정_start','이정_end', '지점번호'])
        data['연도'] = year
    else:
        tmp = pd.DataFrame(np.array([list(map(lambda x: x[0], lst[i])), list(map(lambda x: x[1], lst[i])), num[i]]).T, columns=['이정_start','이정_end', '지점번호'])
        tmp['연도'] = year
        data = pd.concat([data, tmp])

data['지점번호'] = list(map(lambda x: 10 if 100 <= x < 1000 else 500 if 5000 <= x < 9999 else 1000, data['지점번호']))

In [ ]:
ny

In [3]:
data

,이정_start,이정_end,지점번호,연도
0,386.4,391.5,10,2022
1,394.2,407.3,10,2022
2,0.0,4.2,1000,2022
3,9.5,14.5,1000,2022
4,19.5,22.1,1000,2022
5,22.1,25.1,1000,2022
6,72.1,75.2,1000,2022
7,75.2,79.0,1000,2022
8,82.0,84.3,1000,2022
9,84.3,86.4,1000,2022


In [ ]:
def add_categorical_legend(folium_map, title, colors, labels):
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_by_label = dict(zip(labels, colors))
    
    legend_categories = ""     
    for label, color in color_by_label.items():
        legend_categories += f"<li><span style='background:{color}'></span>{label}</li>"
        
    legend_html = f"""
    <div id='maplegend' class='maplegend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legend_categories}
        </ul>
      </div>
    </div>
    """
    script = f"""
        <script type="text/javascript">
        var oneTimeExecution = (function() {{
                    var executed = false;
                    return function() {{
                        if (!executed) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-right').length) || (!executed)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          executed = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        oneTimeExecution()
        </script>
      """
   

    css = """

    <style type='text/css'>
      .maplegend {
        z-index:9999;
        float:right;
        background-color: rgba(255, 255, 255, 1);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        positon: relative;
      }
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    """

    folium_map.get_root().header.add_child(folium.Element(script + css))

    return folium_map

In [3]:
import pandas as pd

In [4]:
RSE = pd.read_csv("road2.csv", encoding='cp949')
RSE.head()

FileNotFoundError: [Errno 2] No such file or directory: 'road2.csv'

In [ ]:
yeongdong_line = RSE[RSE.노선번호 == 500]
yeongdong_line.head()

In [ ]:
seoul_line = RSE[RSE.노선번호 == 1000]
seoul_line.head()

In [ ]:
gyungbu_line = RSE[RSE.노선번호 == 10]
gyungbu_line.head()

In [ ]:
Top = pd.concat([yeongdong_line, seoul_line, gyungbu_line], axis = 0)

In [ ]:
Top = Top[Top['이정']%1 == 0]

In [ ]:
m_1 = folium.Map(location=[37.2,127], tiles='cartodbpositron', zoom_start=9)

def color_producer(val, road, year):
    global data, ny
    
    status = 0
    
    orange = ny.iloc[year-2022, :]
    orange_val_range = list(map(float, orange['이정(km)'].split('-')))
    if orange_val_range[0] <= val <= orange_val_range[1]:
        return 'orange'
        
    target = data[data['연도'] == year]
    for i in range(len(target)):
        if target.iloc[i,0] <= val <= target.iloc[i,1]:
            return 'red'
        
    target = data[data['연도'] < year]
    for i in range(len(target)):
        if target.iloc[i,0] <= val <= target.iloc[i,1]:
            return 'black'
    
    return 'forestgreen'
    

for n in range(2022, 2027):
    idx = 1
    globals()["m_%d"%(idx)] = add_categorical_legend(m_1, '1st Year',
                                 colors = ['#000', '#FF0000', '#FF7F00', '#228B22'],
                               labels = ['Complete',  'Progressing', 'Carry Forward', 'None'])

    # 도로이정 값을 기준으로 100 이하는 초록색, 그외는 빨강색으로 추가했습니다.
    for i in range(0,len(Top)):
        Circle(
            location=[Top.iloc[i]['X좌표값'], Top.iloc[i]['Y좌표값']],
            radius=20,
            color=color_producer(Top.iloc[i]['이정'],Top.iloc[i]['노선번호'], n)).add_to(globals()["m_%d"%(idx)])
    
    idx += 1